# Zadanie 1
Jako przykład rozważmy dana zawierające poziomy przepływu ($\mu g/l$) w trzech grupach pacjentów z zastawką serca z różnymi poziomami wentylacji azotem. W analizie wzięło udział 22 osoby (https://github.com/przem85/statistics/blob/master/D7/ANOVA1.txt).

Zerową hipoteza dla ANOVA mówi, że wszystkie grupy pochodzą z tej samej populacji. 


In [1]:
import numpy as np
import scipy.stats as stats
import pandas as pd

In [2]:
data = np.loadtxt('ANOVA1.txt')
 # Sort them into groups, according to column 1
group1 = data[data[:,1]==1,0]
group2 = data[data[:,1]==2,0]
group3 = data[data[:,1]==3,0]

In [3]:
data

array([[243.,   1.],
       [251.,   1.],
       [275.,   1.],
       [291.,   1.],
       [347.,   1.],
       [354.,   1.],
       [380.,   1.],
       [392.,   1.],
       [206.,   2.],
       [210.,   2.],
       [226.,   2.],
       [249.,   2.],
       [255.,   2.],
       [273.,   2.],
       [285.,   2.],
       [295.,   2.],
       [309.,   2.],
       [241.,   3.],
       [258.,   3.],
       [270.,   3.],
       [293.,   3.],
       [328.,   3.]])

In [4]:
print(group1)
print(group2)
print(group3)

[243. 251. 275. 291. 347. 354. 380. 392.]
[206. 210. 226. 249. 255. 273. 285. 295. 309.]
[241. 258. 270. 293. 328.]


## Saprawdzamy założenie o równości variancii 

In [11]:
# First, check if the variances are equal, with the "Levene"-test
(W,p) = stats.levene(group1, group2, group3)
if p<0.05:
    print(('Warning: the p-value of the Levene test is <0.05: p={0}'.format(p)))

## Wykonujemy ANOVE jednoczynnikową 

In [12]:
# Do the one-way ANOVA
F_statistic, pVal = stats.f_oneway(group1, group2, group3)

In [13]:
print((F_statistic, pVal))
if pVal < 0.05:
    print('One of the groups is significantly different.')

(3.7113359882669763, 0.043589334959178244)
One of the groups is significantly different.


## Bardziej szczegółowym wynikiem ANOVA można otrzymać za pomom modelowania statystycznego:

In [17]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
# Elegant alternative implementation, with pandas & statsmodels
df = pd.DataFrame(data, columns=['value', 'treatment'])    
#df

In [16]:
model = ols('value ~ C(treatment)', df).fit()
anovaResults = anova_lm(model)
print(anovaResults)

                df        sum_sq      mean_sq         F    PR(>F)
C(treatment)   2.0  15515.766414  7757.883207  3.711336  0.043589
Residual      19.0  39716.097222  2090.320906       NaN       NaN


C:\Users\przem85\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\przem85\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\przem85\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


# Możemy policzyć ten test na piechotę

In [31]:
data = np.loadtxt('F:\SZKOLENIE\data\ANOVA1.txt')

In [32]:
# Convert them to pandas-forman and group them by their group value
df = pd.DataFrame(data, columns=['values', 'group'])
groups = df.groupby('group')

Najpierw oblicza się Sums of squares ($SS_{Total}$). Otrzymujemy 

$$
SS_{Treatments}=15 515.76
$$ 

oraz 

$$
SS_{Error}=39.716.09.
$$

In [38]:
# The "total sum-square" is the squared deviation from the mean
ss_total = np.sum((df['values']-df['values'].mean())**2)
ss_total

55231.86363636363

In [44]:
# Calculate ss_treatment and  ss_error
(ss_treatments, ss_error) = (0, 0)
for val, group in groups:
    ss_error += sum((group['values'] - group['values'].mean())**2)
    ss_treatments += len(group) * (group['values'].mean() - df['values'].mean())**2
print(ss_treatments)
print(ss_error)

15515.766414141408
39716.0972222


Średnie kwadraty (mean squares) (MS) to SS podzielone przez odpowiednie stopnie swobody (DF).

In [46]:
df_groups = len(groups)-1
df_residuals = len(data)-len(groups)

Otrzymujemy również wartość statystyki $F$:
$$
F=\frac{MS_{Treatments}}{MS_{Error}}=\frac{SS_{Treatments}/(n_{groups}-1)}{MS_{Error}/(n_{total}-n_{groups})}
$$

In [47]:
F = (ss_treatments/df_groups) / (ss_error/df_residuals)
df = stats.f(df_groups,df_residuals)
p = df.sf(F)
print(('ANOVA-Results: F = {0}, and p<{1}'.format(F, p)))

ANOVA-Results: F = 3.7113359882669754, and p<0.043589334959178244
